# Automação Web Scraping

### Automatizar consultas de cotação de todas as commodites de um banco de dados em um site, retornar o valor atual,comparar com o antigo e verificar se vale a pena comprar ou não

### Usaremos as bibliotecas:
- Selenium
- Pandas
- Unidecode

In [27]:
#Passo 1- Iniciando o navegador

from selenium import webdriver

navegador = webdriver.Chrome()

navegador.get("https://www.google.com/")



In [28]:
#Passo 2- Importando base de dados
import pandas as pd

#Lendo arquivo Excel
tabela = pd.read_excel("commodities.xlsx")

display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Tilápia,9.05,NaN,NaN


In [29]:
#Passo 3- Concultando cada produto no site

import unicodedata

#Estrutura para repetir para cada item da tabela
for linha in tabela.index:
    
    #recebendo nome do produto da linha correspondente
    produto = tabela.loc[linha,"Produto"]
    
    #Fazendo a consulta com o nome do produto
    
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    
    #Código para tirar todos os sinais para evitar erros
    
    link = unicodedata.normalize("NFKD",link).encode("ascii","ignore").decode("utf-8")
    
    navegador.get(link)
    
    #Armazenando o preço do produto atravez do seu elemento do site
    cotacao = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
    
    #Removendo sinais e transformando o valor de texto para numérico
    cotacao = cotacao.replace(".","").replace(",",".")
    cotacao = float(cotacao)
    
    #Armazenando o valor na tabela na mesma linha do produto correspondente
    tabela.loc[linha,"Preço Atual"] = cotacao
    

#Visualizando tabela
print("Finalizado")
display(tabela)





Finalizado


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,56.51,NaN
1,Soja,163.59,132.56,NaN
2,Boi,282.20,267.30,NaN
3,Petróleo,424.37,382.45,NaN
4,Algodão,497.76,395.63,NaN
5,Açúcar,136.23,148.83,NaN
6,Café,1092.87,1043.39,NaN
7,Ouro,321.77,315.20,NaN
8,Tilápia,9.05,9.64,NaN


In [30]:
#Passo 4- Comparando os preços e se deve comprar ou não

tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]

display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,56.51,True
1,Soja,163.59,132.56,True
2,Boi,282.20,267.30,True
3,Petróleo,424.37,382.45,True
4,Algodão,497.76,395.63,True
5,Açúcar,136.23,148.83,False
6,Café,1092.87,1043.39,True
7,Ouro,321.77,315.20,True
8,Tilápia,9.05,9.64,False


In [31]:
#fechando navegador
navegador.quit()

In [32]:
#Passo 5- Exportando a base de dados atualizada

tabela.to_excel("Commodites_novos.xlsx",index=False)